# For our contrarian algorithm, we assume that the most recent (extreme) prices will revert in the near future to their standard price.

### We use yfinance (Yahoo Finance) for stock data and IKBR as a broker. We run this at the end of the trading day.

In [ ]:
# We start by importing all libaries.

In [ ]:
import pandas as pd
import yfinance as yf
from ib_insync import *
util.startLoop()

In [ ]:
# We set our parameters to use later

In [ ]:
top_stocks = 3 # The top N stocks that increased the most
bottom_stocks = 3 # The bottom N stocks that decreased the most
shares = 1 # How many shares we purchase of each stock

In [ ]:
# Next, we pull the tickers in the DJIA index.

In [ ]:
url = "https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average"
df = pd.read_html(url)[1]
df.set_index("Symbol", inplace = True)
tickers = df.index.to_list()

In [ ]:
# We gather the percentage change in our chosen stocks.

In [ ]:
stock_changes = pd.Series(dtype = float)

for ticker in tickers:
    current_close = yf.Ticker(ticker = ticker).fast_info["last_price"]
    past_close = yf.Ticker(ticker = ticker).fast_info["previous_close"]
    percentage_change = current_close / past_close - 1
    stock_changes.loc[ticker] = percentage_change
    print("{}/{}".format(tickers.index(ticker)+1, len(tickers)), end = '\r')

In [ ]:
# We find the 3 biggest percentage increases and assign a + share amount (denoting buy)

In [ ]:
stock_changes.sort_values(inplace = True, ascending= False)
sorted_descending_list = stock_changes[:top_stocks]
sorted_descending_list.iloc[:] = shares

In [ ]:
# We find the 3 smallest percentage increases and assign a - share amount (denoting sell)

In [ ]:
stock_changes.sort_values(inplace = True, ascending= True)
sorted_ascending_list = stock_changes[:bottom_stocks]
sorted_ascending_list.iloc[:] = -shares

In [ ]:
# We amalgamate the list

In [ ]:
trade_list = pd.concat([sorted_descending_list, sorted_ascending_list]).to_frame().reset_index()
trade_list.columns = ["symbol", "position"]

In [ ]:
# We connect to Interactive Brokers

In [ ]:
ib = IB()
ib.connect()

In [ ]:
# We check our current positions (if we already hold a stock, we don't need to repurchase it)

In [ ]:
current_positions = util.df(ib.positions())

In [ ]:
# We create an empty data frame if there is no data to pull from (in the case that current_positions is empty)

In [ ]:
if current_positions is not None:
    current_positions["symbol"] = current_positions.contract.apply(lambda x: x.symbol)
    current_positions["conID"] = current_positions.contract.apply(lambda x: x.conId)
else: 
    current_positions = pd.DataFrame(columns = ["symbol", "position"])

In [ ]:
# We merge our current_positions with our trade_list to see which stocks we need to buy or sell

In [ ]:
trades = pd.merge(trade_list, current_positions[["symbol", "position"]], "outer", on = "symbol", suffixes = ["_totrade", "_current"])
trades.fillna(0, inplace = True)
trades["trades"] = trades.position_totrade - trades.position_current
final_trades = trades[trades.trades !=0].set_index("symbol").copy()

In [ ]:
# We run a loop to execute our trades (if applicable)

In [ ]:
for symbol in final_trades.index: # We loop through our ticker symbols
    
    to_trade = final_trades.loc[symbol, "trades"] # We read if we buy, sell, or do nothing for our current ticker
    if to_trade > 0: 
        side = "BUY"
    elif to_trade < 0:
        side = "SELL"
    else:
        continue

    contract = Stock(symbol, "SMART", "USD") # We create a contract for our ticker symbol
    cds = ib.reqContractDetails(contract) # We pull information from the contract
    if len(cds) == 0:
        print("No Contract for {} found.".format(symbol))
    else:
        contract = cds[0].contract # We pull the first contract (if there are one or multiple contracts)
    
        order = MarketOrder(side, abs(to_trade)) # We create our order
        trade = ib.placeOrder(contract, order) # We place our order
        
        while not trade.isDone(): # We add a pause to wait for our order to go through
            ib.waitOnUpdate()
        
        if trade.orderStatus.status == "Filled": # We return a trade status update
            print("{} {} @ {}".format(side, symbol, trade.orderStatus.avgFillPrice))
        else:
            print("{} {} failed.".format(side, symbol))

In [ ]:
# We recheck our current positions to confirm that our trades have executed

In [ ]:
current_positions = util.df(ib.positions())
current_positions["symbol"] = current_positions.contract.apply(lambda x: x.symbol)
current_positions["conID"] = current_positions.contract.apply(lambda x: x.conId)
current_positions

In [ ]:
# We disconnect from Interactive Brokers

In [ ]:
ib.disconnect()